# Perceptual Loss For Real Time Transfer

该网络与原始版本的Style Transfer的套路是一样的，唯一的不同就是原始版本的style transfer是直接拿vgg-16，从一张噪音图开始，一方面比对content image，另一方面比对style image，不断调整噪音图，从而得到一张融合后的图。而Percept Loss For Real Time Transfer则是增加了一个auto encoder形状的网络，叫做transformation network，进原图出来预测图，这个预测图与原图在内容做比对，与风格图在风格上做比对。在训练的时候，有若干张内容图，只有一张风格图。这样就可以训练transformation network一边顾及到恢复原图，一边顾及到贴近特定的风格。  
结果就是，一个transformation network就贴近一张风格图。这样一来，一张新的内容图进来以后，直接就可以推断出风格迁移后的图像。图像质量理论上跟原始版本style transfer不会有啥差异，只不过是可以提前训练好该模型，不像style transfer原始版本，每次都要训练。

首先，拿一系列内容图+一张风格图训练一个网络，并保存下来

In [1]:
from main import build_parser
from train import network_train
from test import network_test
import os
from pathlib import Path

# 当前目录
#current_directory = os.path.dirname(os.path.abspath(__file__))
current_directory = os.getcwd()


d:\Program Files\Anaconda3\envs\pytorchgpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 训练模式
content_imagedir = os.path.join(current_directory, "../Images/original/")
style_imagedir = os.path.join(current_directory, "../Images/cartoon/")
content_weight = 10
style_weight = 500
tv_weight = 3
max_iter = 4000
subdirname = "Args" + str(content_weight) + "_" + str(style_weight) + "_" + str(tv_weight) + "_" + str(max_iter) + "/"

model_outputpath = os.path.join(current_directory, "trained_models/", subdirname)

style_files = os.listdir(style_imagedir)
for style_filename in style_files:
    parser = build_parser()
    style_imagepath = os.path.join(style_imagedir, style_filename)
    stylename_nosuffix = str(Path(style_filename).with_suffix("")) 
    # 当前模型输出目录
    curmodel_outputdir = os.path.join(model_outputpath, stylename_nosuffix, "./")
    print("OutputDir:" + curmodel_outputdir)
    # 创建该目录
    is_exist = os.path.exists(curmodel_outputdir)
    if not is_exist:
        os.makedirs(curmodel_outputdir)

    #style_imagepath = os.path.join(current_directory, "../Images/cartoon/0001.jpg")
    custom_args = [
        "--train-flag", "True",
        "--train-content", content_imagedir,
        "--train-style", style_imagepath,
        "--content-weight", str(content_weight),
        "--style-weight", str(style_weight),
        "--tv-weight", str(tv_weight),
        "--max-iter", str(max_iter),
        "--save-path", curmodel_outputdir
    ]
    args = parser.parse_args(custom_args)
    print(args)
    network_train(args)


OutputDir:e:\workspace\577-project\PerceptualLossForRealTimeTransfer\trained_models/Args10_500_3_4000/0000\./
Namespace(batchs=8, check_iter=100, content_layers=[15], content_weight=10.0, cropsize=None, cuda_device_no=0, imsize=256, lr=0.1, max_iter=4000, model_load_path=None, output='stylized.jpg', save_path='e:\\workspace\\577-project\\PerceptualLossForRealTimeTransfer\\trained_models/Args10_500_3_4000/0000\\./', style_layers=[3, 8, 15, 22], style_weight=500.0, test_content=None, train_content='e:\\workspace\\577-project\\PerceptualLossForRealTimeTransfer\\../Images/original/', train_flag=True, train_style='e:\\workspace\\577-project\\PerceptualLossForRealTimeTransfer\\../Images/cartoon/0000.jpg', tv_weight=3.0, vgg_flag='vgg16')
device:cuda:0


d:\Program Files\Anaconda3\envs\pytorchgpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Program Files\Anaconda3\envs\pytorchgpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Wed Nov 22 20:51:37 2023: iteration: [0/4000/],	content_loss: 0.22,	style_loss: 0.48,	tv_loss: 0.01,	total_loss: 243.44,	
Wed Nov 22 20:52:52 2023: iteration: [100/4000/],	content_loss: 10.21,	style_loss: 2.28,	tv_loss: 0.04,	total_loss: 1242.55,	
Wed Nov 22 20:54:57 2023: iteration: [200/4000/],	content_loss: 12.27,	style_loss: 1.75,	tv_loss: 0.08,	total_loss: 996.05,	
Wed Nov 22 20:57:17 2023: iteration: [300/4000/],	content_loss: 13.57,	style_loss: 1.38,	tv_loss: 0.10,	total_loss: 827.24,	
Wed Nov 22 20:59:31 2023: iteration: [400/4000/],	content_loss: 14.53,	style_loss: 1.12,	tv_loss: 0.12,	total_loss: 706.71,	
Wed Nov 22 21:00:37 2023: iteration: [500/4000/],	content_loss: 15.27,	style_loss: 0.92,	tv_loss: 0.13,	total_loss: 612.38,	
Wed Nov 22 21:01:43 2023: iteration: [600/4000/],	content_loss: 15.64,	style_loss: 0.78,	tv_loss: 0.14,	total_loss: 547.60,	
Wed Nov 22 21:02:57 2023: iteration: [700/4000/],	content_loss: 15.92,	style_loss: 0.69,	tv_loss: 0.14,	total_loss: 504.61,	
We

现在开始测试

In [ ]:
parser = build_parser()
model_path = os.path.join(current_directory, "trained_models/transform_network.pth")
test_imagepath = os.path.join(current_directory, "../Images/content-images/HeadBase.jpg")
output_imagepath = os.path.join(current_directory, "../Images/styletransfer-output/HeadBaseTransfered.jpg")
custom_args = [
    "--train-flag", "False",
    "--model-load-path", model_path,
    "--test-content", test_imagepath,
    "--output", output_imagepath
]

args = parser.parse_args(custom_args)
print(args)

network_test(args)
